### Relevancia Marginal Máxima (MMR - Maximal Marginal Relevance)

**MMR (Maximal Marginal Relevance)** es una técnica poderosa de recuperación consciente de la diversidad utilizada en recuperación de información y pipelines RAG para equilibrar relevancia y novedad al seleccionar documentos.

**¿Qué problema resuelve MMR?**

Los métodos tradicionales de búsqueda por similitud pueden devolver documentos muy similares entre sí (redundantes). MMR aborda este problema seleccionando documentos que son:
1. **Relevantes** a la consulta del usuario
2. **Diversos** entre sí (minimizan la redundancia)

**¿Cómo funciona?**

MMR utiliza una fórmula que balancea dos objetivos:
- **Similitud con la consulta**: Qué tan relevante es el documento para la pregunta
- **Disimilitud con documentos ya seleccionados**: Qué tan diferente es de los documentos ya recuperados

**Ventajas:**
- ✅ Evita información repetitiva
- ✅ Proporciona una perspectiva más amplia del tema
- ✅ Mejora la calidad de las respuestas al incluir información complementaria
- ✅ Especialmente útil cuando hay múltiples aspectos de una pregunta

In [1]:
# Importación de librerías necesarias para implementar MMR

# FAISS: Librería de Facebook para búsqueda de similitud vectorial eficiente
from langchain_community.vectorstores import FAISS

# HuggingFaceEmbeddings: Para generar embeddings usando modelos de HuggingFace
from langchain_huggingface import HuggingFaceEmbeddings

# TextLoader: Para cargar archivos de texto plano
from langchain.document_loaders import TextLoader

# RecursiveCharacterTextSplitter: Para dividir texto en fragmentos manejables
from langchain.text_splitter import RecursiveCharacterTextSplitter

# init_chat_model: Para inicializar modelos de chat de diferentes proveedores
from langchain.chat_models import init_chat_model

# PromptTemplate: Para crear plantillas de prompts con variables dinámicas
from langchain.prompts import PromptTemplate

# create_stuff_documents_chain: Crea una cadena que inserta documentos en un prompt
from langchain.chains.combine_documents import create_stuff_documents_chain

# create_retrieval_chain: Combina recuperación de documentos con generación de respuestas
from langchain.chains.retrieval import create_retrieval_chain

c:\Udemy\RAGBootcamp\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Configuración de variables de entorno para las API keys

# Importar módulos para manejo de variables de entorno
import os
from dotenv import load_dotenv

# Cargar las variables de entorno desde el archivo .env
load_dotenv()

# Establecer la API key de OpenAI desde las variables de entorno
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

# Establecer la API key de Groq desde las variables de entorno
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [3]:
# Paso 1: Cargar y fragmentar el documento

# Cargar el archivo de texto que contiene información sobre LangChain y RAG
loader = TextLoader("langchain_rag_dataset.txt")
# load() devuelve una lista de documentos con todo el contenido del archivo
raw_docs = loader.load()

# Crear un splitter para dividir el texto en fragmentos más pequeños
# chunk_size=300: Cada fragmento tendrá máximo 300 caracteres
# chunk_overlap=50: Habrá una superposición de 50 caracteres entre fragmentos consecutivos
# La superposición ayuda a mantener el contexto entre fragmentos
splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)

# Dividir los documentos crudos en fragmentos (chunks) manejables
chunks = splitter.split_documents(raw_docs)

# Mostrar los fragmentos generados
chunks

[Document(metadata={'source': 'langchain_rag_dataset.txt'}, page_content='LangChain es un framework de cÃ³digo abierto diseÃ±ado para simplificar el desarrollo de aplicaciones que utilizan grandes modelos de lenguaje (LLM).'),
 Document(metadata={'source': 'langchain_rag_dataset.txt'}, page_content='LangChain proporciona abstracciones para trabajar con indicaciones, cadenas, memoria y agentes, lo que facilita la creaciÃ³n de sistemas complejos basados â€‹â€‹en LLM.\nEl framework admite la integraciÃ³n con diversas bases de datos vectoriales como FAISS y Chroma para la recuperaciÃ³n semÃ¡ntica.'),
 Document(metadata={'source': 'langchain_rag_dataset.txt'}, page_content='LangChain habilita la GeneraciÃ³n Aumentada por RecuperaciÃ³n (RAG), permitiendo a los desarrolladores obtener el contexto relevante antes de generar respuestas.'),
 Document(metadata={'source': 'langchain_rag_dataset.txt'}, page_content='La memoria en LangChain ayuda a los modelos a retener interacciones previas, lo que

In [4]:
# Paso 2: Crear el almacén vectorial FAISS con embeddings de HuggingFace

# Inicializar el modelo de embeddings de HuggingFace
# all-MiniLM-L6-v2 es un modelo compacto y eficiente que genera vectores de 384 dimensiones
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Crear el almacén vectorial FAISS a partir de los fragmentos de documentos
# FAISS convierte cada fragmento en un vector numérico para búsqueda de similitud eficiente
# Este almacén soportará búsqueda MMR para diversidad en los resultados
vectorstore = FAISS.from_documents(chunks, embedding_model)

In [5]:
# Paso 3: Crear el recuperador con MMR (Maximal Marginal Relevance)

# Convertir el almacén vectorial en un recuperador configurado para usar MMR
retriever = vectorstore.as_retriever(
    search_type="mmr",  # Especifica que usaremos MMR en lugar de similitud simple
                        # MMR balancea relevancia y diversidad en los resultados
    search_kwargs={"k": 3}  # k=3: Recuperar los 3 documentos más relevantes y diversos
                            # MMR seleccionará documentos que sean relevantes pero también
                            # diferentes entre sí para evitar redundancia
)

In [ ]:
# Celda vacía - puede ser usada para pruebas o código adicional

In [6]:
# Paso 4: Configurar el prompt y el modelo de lenguaje (LLM)

# Crear la plantilla de prompt que define cómo el LLM usará el contexto recuperado
prompt = PromptTemplate.from_template("""
Responde la pregunta basándote en el contexto proporcionado.

Contexto:
{context}

Pregunta: {input}
""")

# Inicializar el LLM usando Groq con el modelo Gemma2-9B
# Groq ofrece inferencia rápida y Gemma2 es eficiente para generación de texto
llm = init_chat_model("groq:llama-3.1-8b-instant")

In [7]:
# Paso 5: Construir el pipeline RAG completo

# Crear la cadena de documentos que combina el LLM con el prompt
# Esta cadena toma documentos recuperados y los inserta en el prompt
document_chain = create_stuff_documents_chain(llm=llm, prompt=prompt)

# Crear la cadena RAG completa que integra:
# 1. El recuperador MMR (para obtener documentos relevantes y diversos)
# 2. La cadena de documentos (para generar la respuesta con el LLM)
rag_chain = create_retrieval_chain(retriever=retriever, combine_docs_chain=document_chain)

In [8]:
# Paso 6: Realizar una consulta al sistema RAG con MMR

# Definir la pregunta que queremos responder
# Esta pregunta toca dos temas diferentes: agentes y memoria
# MMR ayudará a recuperar documentos diversos que cubran ambos aspectos
query = {"input": "¿Cómo soporta LangChain los agentes y la memoria?"}

# Invocar la cadena RAG completa con nuestra consulta
# El flujo será: consulta → recuperación MMR → inserción en prompt → generación LLM
response = rag_chain.invoke(query)

# Mostrar solo la respuesta generada por el LLM
print("✅ Respuesta:\n", response["answer"])

✅ Respuesta:
 Según el contexto proporcionado, LangChain soporta a los agentes y la memoria de las siguientes maneras:

- La memoria en LangChain ayuda a los modelos a retener interacciones previas, lo que aumenta la coherencia de las conversaciones multi-turno.
- Los agentes en LangChain pueden usar herramientas como calculadoras, API de búsqueda o funciones personalizadas según las instrucciones que reciben.
- LangChain permite la reclasificación de los resultados recuperados mediante LLM o codificadores cruzados neuronales para mejorar la calidad del contexto.


In [9]:
# Mostrar la respuesta completa incluyendo contexto y metadatos
# Esta celda muestra:
# - 'input': La pregunta original
# - 'context': Los documentos recuperados por MMR (3 documentos diversos)
# - 'answer': La respuesta generada por el LLM basada en el contexto

# Nota: Observa que MMR seleccionó 3 documentos diferentes que cubren aspectos
# complementarios: memoria en conversaciones, agentes con APIs, y tipos de memoria
response

{'input': '¿Cómo soporta LangChain los agentes y la memoria?',
 'context': [Document(id='c5deca15-ebf4-4933-92b2-a4584cb1e0d3', metadata={'source': 'langchain_rag_dataset.txt'}, page_content='La memoria en LangChain ayuda a los modelos a retener interacciones previas, lo que aumenta la coherencia de las conversaciones multi-turno.\nLos agentes en LangChain pueden usar herramientas como calculadoras, API de bÃºsqueda o funciones personalizadas segÃºn las instrucciones que reciben.'),
  Document(id='38c71fd4-e659-4086-a7a8-cbc01ff03262', metadata={'source': 'langchain_rag_dataset.txt'}, page_content='de respuestas basadas en LLM (relevancia marginal mÃ¡xima) de documentos.'),
  Document(id='ad40c814-be7a-4065-8874-dc10b6ef2575', metadata={'source': 'langchain_rag_dataset.txt'}, page_content='LangChain permite la reclasificaciÃ³n de los resultados recuperados mediante LLM o codificadores cruzados neuronales para mejorar la calidad del contexto.')],
 'answer': 'Según el contexto proporcion

In [ ]:
# Celda vacía - puede ser usada para experimentos adicionales
# Por ejemplo, podrías:
# - Probar diferentes valores de k en el recuperador MMR
# - Comparar resultados de MMR vs búsqueda por similitud simple
# - Ajustar el tamaño de los chunks para ver cómo afecta los resultados